# Cross Validation to find optimal latent rank (and hyperparamters)

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import numba
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, Predictor, als, basic, user_knn
from lenskit import topn
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from lenskit.data import sparse_ratings

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations and debugging
import plotly.graph_objs as go
#from pprintpp import pprint as pp
import logging

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

In [2]:
from scipy.sparse.linalg import svds, eigs, norm
from scipy.sparse import csr_matrix
from numpy import linalg as LA
from numba import jit, njit, prange

In [3]:
# Dataset
ratings = pd.read_parquet('ratings.parquet.gzip')

In [ ]:
ratings

In [4]:
@jit(nopython=True)
def matrix_factorization_pred(X, P, Q, K, steps, alpha, beta, Mask):
    #    Mask = (X!=0)
    Q = Q.T
    error_list = np.zeros(steps+1)
    error_list[0] = 100000000
    for step in range(steps):
        # for each user
        for i in prange(X.shape[0]):
            # for each item
            for j in range(X.shape[1]):
                if X[i, j] > 0:

                    # calculate the error of the element
                    eij = X[i, j] - np.dot(P[i, :], Q[:, j])
                    # second norm of P and Q for regularilization
                    sum_of_norms = 0
                    # for k in xrange(K):
                    #    sum_of_norms += LA.norm(P[:,k]) + LA.norm(Q[k,:])
                    # added regularized term to the error
                    sum_of_norms += LA.norm(P) + LA.norm(Q)
                    # print sum_of_norms
                    eij += (beta / 2) * sum_of_norms
                    # compute the gradient from the error
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (
                            2 * eij * Q[k][j] - (beta * P[i][k])
                        )
                        Q[k][j] = Q[k][j] + alpha * (
                            2 * eij * P[i][k] - (beta * Q[k][j])
                        )

        # compute total error
        error = 0
        # for each user
        extimated_X = np.trunc(P @ Q)
        extimated_X = np.where(extimated_X > 5, 5, extimated_X)
        extimated_X = np.where(extimated_X < 0, 0, extimated_X)
        extimated_error = np.multiply(X - extimated_X, Mask)
        err_temp = LA.norm(extimated_error)
        error_list[step+1] = err_temp
        
        print(step)

        if np.abs(err_temp - error_list[step]) < 0.001:
            break
    return extimated_X, P, Q.T, error_list

In [5]:
# <script src="https://gist.github.com/tgsmith61591/ce7d614d7a0442f94cd5ae5d1e51d3c2.js"></script>

# -*- coding: utf-8 -*-
#
# Author: Taylor G Smith
#
# More scratch code in my collection of random recommender
# system utilities. Someday I'll get around to building
# an actual repository... in the meantime, here are some
# train/test split utilities for collaborative filtering
# with sparse matrices.

from __future__ import absolute_import, division

import numpy as np
from abc import ABCMeta, abstractmethod

import six
from sklearn.utils.validation import check_random_state
from sklearn.utils import validation as skval

from scipy import sparse

import numbers

from sklearn.preprocessing import LabelEncoder

__all__ = [
    'BootstrapCV',
    'check_cv',
    'train_test_split'
]

MAX_SEED = 1e6
ITYPE = np.int32
DTYPE = np.float64  # implicit asks for doubles, not float32s...


def check_consistent_length(u, i, r):
    """Ensure users, items, and ratings are all of the same dimension.
    Parameters
    ----------
    u : array-like, shape=(n_samples,)
        A numpy array of the users.
    i : array-like, shape=(n_samples,)
        A numpy array of the items.
    r : array-like, shape=(n_samples,)
        A numpy array of the ratings.
    """
    skval.check_consistent_length(u, i, r)
    return np.asarray(u), np.asarray(i), np.asarray(r, dtype=DTYPE)


def _make_sparse_csr(data, rows, cols, dtype=DTYPE):
    # check lengths
    check_consistent_length(data, rows, cols)
    data, rows, cols = (np.asarray(x) for x in (data, rows, cols))

    shape = (np.unique(rows).shape[0], np.unique(cols).shape[0])
    return sparse.csr_matrix((data, (rows, cols)),
                             shape=shape, dtype=dtype)


def to_sparse_csr(u, i, r, axis=0, dtype=DTYPE):
    """Create a sparse ratings matrix.
    Create a sparse ratings matrix with users and items as rows and columns,
    and ratings as the values.
    Parameters
    ----------
    u : array-like, shape=(n_samples,)
        The user vector. Positioned along the row axis if ``axis=0``,
        otherwise positioned along the column axis.
    i : array-like, shape=(n_samples,)
        The item vector. Positioned along the column axis if ``axis=0``,
        otherwise positioned along the row axis.
    r : array-like, shape=(n_samples,)
        The ratings vector.
    axis : int, optional (default=0)
        The axis along which to position the users. If 0, the users are
        along the rows (with items as columns). If 1, the users are columns
        with items as rows.
    dtype : type, optional (default=np.float32)
        The type of the values in the ratings matrix.
    """
    if axis not in (0, 1):
        raise ValueError("axis must be an int in (0, 1)")

    rows = u if axis == 0 else i
    cols = i if axis == 0 else u
    return _make_sparse_csr(data=r, rows=rows, cols=cols, dtype=dtype)


def check_cv(cv=3):
    """Input validation for cross-validation classes.
    Parameters
    ----------
    cv : int, None or BaseCrossValidator
        The CV class or number of folds.
        - None will default to 3-fold BootstrapCV
        - integer will default to ``integer``-fold BootstrapCV
        - BaseCrossValidator will pass through untouched
    Returns
    -------
    checked_cv : BaseCrossValidator
        The validated CV class
    """
    if cv is None:
        cv = 3

    if isinstance(cv, numbers.Integral):
        return BootstrapCV(n_splits=int(cv))
    if not hasattr(cv, "split") or isinstance(cv, six.string_types):
        raise ValueError("Expected integer or CV class, but got %r (type=%s)"
                         % (cv, type(cv)))
    return cv


def _validate_train_size(train_size):
    """Train size should be a float between 0 and 1."""
    assert isinstance(train_size, float) and (0. < train_size < 1.), \
        "train_size should be a float between 0 and 1"


def _get_stratified_tr_mask(u, i, train_size, random_state):
    _validate_train_size(train_size)  # validate it's a float
    random_state = check_random_state(random_state)
    n_events = u.shape[0]

    # this is our train mask that we'll update over the course of this method
    train_mask = random_state.rand(n_events) <= train_size  # type: np.ndarray

    # we have a random mask now. For each of users and items, determine which
    # are missing from the mask and randomly select one of each of their
    # ratings to force them into the mask
    for array in (u, i):
        # e.g.:
        # >>> array = np.array([1, 2, 3, 3, 1, 3, 2])
        # >>> train_mask = np.array([0, 1, 1, 1, 0, 0, 1]).astype(bool)
        # >>> unique, counts = np.unique(array, return_counts=True)
        # >>> unique, counts
        # (array([1, 2, 3]), array([2, 2, 3]))

        # then present:
        # >>> present
        # array([2, 3, 3, 2])
        present = array[train_mask]

        # and the test indices:
        # >>> test_vals
        # array([1, 1, 3])
        test_vals = array[~train_mask]

        # get the test indices that are NOT present (either
        # missing items or users)
        # >>> missing
        # array([1])
        missing = np.unique(test_vals[np.where(
            ~np.in1d(test_vals, present))[0]])

        # If there is nothing missing, we got perfectly lucky with our random
        # split and we'll just go with it...
        if missing.shape[0] == 0:
            continue

        # Otherwise, if we get to this point, we have to add in the missing
        # level to the mask to make sure at least one of each of those makes
        # it into the training data (so we don't lose a factor level for ALS)
        array_mask_missing = np.in1d(array, missing)

        # indices in "array" where we have a level that's currently missing
        # and that needs to be added into the mask
        where_missing = np.where(array_mask_missing)[0]  # e.g., array([0, 4])

        # I don't love having to loop here... but we'll iterate "where_missing"
        # to incrementally add in items or users until all are represented
        # in the training set to some degree
        added = set()
        for idx, val in zip(where_missing, array[where_missing]):
            # if we've already seen and added this one
            if val in added:  # O(1) lookup
                continue

            train_mask[idx] = True
            added.add(val)

    return train_mask


def _make_sparse_tr_te(users, items, ratings, train_mask):
    # now make the sparse matrices
    r_train = to_sparse_csr(u=users[train_mask], i=items[train_mask],
                            r=ratings[train_mask], axis=0)

    # TODO: anti mask for removing from test set?
    r_test = to_sparse_csr(u=users, i=items, r=ratings, axis=0)
    return r_train, r_test


def train_test_split(u, i, r, train_size=0.75, random_state=None):
    """Create a train/test split for sparse ratings.
    Given vectors of users, items, and ratings, create a train/test split
    that preserves at least one of each user and item in the training split
    to prevent inducing a cold-start situation.
    Parameters
    ----------
    u : array-like, shape=(n_samples,)
        A numpy array of the users. This vector will be used to stratify the
        split to ensure that at least of each of the users will be included
        in the training split. Note that this diminishes the likelihood of a
        perfectly-sized split (i.e., ``len(train)`` may not exactly equal
        ``train_size * n_samples``).
    i : array-like, shape=(n_samples,)
        A numpy array of the items. This vector will be used to stratify the
        split to ensure that at least of each of the items will be included
        in the training split. Note that this diminishes the likelihood of a
        perfectly-sized split (i.e., ``len(train)`` may not exactly equal
        ``train_size * n_samples``).
    r : array-like, shape=(n_samples,)
        A numpy array of the ratings.
    train_size : float, optional (default=0.75)
        The ratio of the train set size. Should be a float between 0 and 1.
    random_state : RandomState, int or None, optional (default=None)
        The random state used to create the train mask.
    Examples
    --------
    An example of a sparse matrix split that masks some ratings from the train
    set, but not from the testing set:
    >>> u = [0, 1, 0, 2, 1, 3]
    >>> i = [1, 2, 2, 0, 3, 2]
    >>> r = [0.5, 1.0, 0.0, 1.0, 0.0, 1.]
    >>> train, test = train_test_split(u, i, r, train_size=0.5,
    ...                                random_state=42)
    >>> train.toarray()
    array([[ 0. ,  0.5,  0. ,  0. ],
           [ 0. ,  0. ,  0. ,  0. ],
           [ 1. ,  0. ,  0. ,  0. ],
           [ 0. ,  0. ,  1. ,  0. ]], dtype=float32)
    >>> test.toarray()
    array([[ 0. ,  0.5,  0. ,  0. ],
           [ 0. ,  0. ,  1. ,  0. ],
           [ 1. ,  0. ,  0. ,  0. ],
           [ 0. ,  0. ,  1. ,  0. ]], dtype=float32)
    Here's a more robust example (with more ratings):
    >>> from sklearn.preprocessing import LabelEncoder
    >>> import numpy as np
    >>> rs = np.random.RandomState(42)
    >>> users = np.arange(100000)  # 100k users in DB
    >>> items = np.arange(30000)  # 30k items in DB
    >>> # Randomly select some for ratings:
    >>> items = rs.choice(items, users.shape[0])  # 100k rand item rtgs
    >>> users = rs.choice(users, users.shape[0])  # 100k rand user rtgs
    >>> # Label encode so they're positional indices:
    >>> users = LabelEncoder().fit_transform(users)
    >>> items = LabelEncoder().fit_transform(items)
    >>> ratings = rs.choice((0., 0.25, 0.5, 0.75, 1.), items.shape[0])
    >>> train, test = train_test_split(users, items, ratings, random_state=rs)
    >>> train
    <26353x28921 sparse matrix of type '<type 'numpy.float32'>'
        with 77770 stored elements in Compressed Sparse Row format>
    >>> test
    <26353x28921 sparse matrix of type '<type 'numpy.float32'>'
        with 99994 stored elements in Compressed Sparse Row format>
        
    Notes
    -----
    ``u``, ``i`` inputs should be encoded (i.e., via LabelEncoder) prior to
    splitting the data. This is due to the indexing behavior used within the
    function.
    Returns
    -------
    r_train : scipy.sparse.csr_matrix
        The train set.
    r_test : scipy.sparse.csr_matrix
        The test set.
    """
    # make sure all of them are numpy arrays and of the same length
    users, items, ratings = check_consistent_length(u, i, r)

    train_mask = _get_stratified_tr_mask(
        users, items, train_size=train_size,
        random_state=random_state)

    return _make_sparse_tr_te(users, items, ratings, train_mask=train_mask)


# avoid pb w nose
train_test_split.__test__ = False


class BaseCrossValidator(six.with_metaclass(ABCMeta)):
    """Base class for all collab CV.
    Iterations must define ``_iter_train_mask``. This is based loosely
    on sklearn's cross validator but does not adhere to its exact
    interface.
    """

    def __init__(self, n_splits=3, random_state=None):
        self.n_splits = n_splits
        self.random_state = random_state

    def get_n_splits(self):
        return self.n_splits

    def split(self, X):
        """Generate indices to split data into training and test sets.
        Parameters
        ----------
        X : scipy.sparse.csr_matrix
            A sparse ratings matrix.
        Returns
        -------
        train : scipy.sparse.csr_matrix
            The training set
        test : scipy.sparse.csr_matrix
            The test set
        """
        ratings = X.data
        users, items = X.nonzero()

        # make sure all of them are numpy arrays and of the same length
        # users, items, ratings = check_consistent_length(u, i, r)
        for train_mask in self._iter_train_mask(users, items, ratings):
            # yield in a generator so we don't have to store in mem
            yield _make_sparse_tr_te(users, items, ratings,
                                     train_mask=train_mask)

    @abstractmethod
    def _iter_train_mask(self, u, i, r):
        """Compute the training mask here.
        Returns
        -------
        train_mask : np.ndarray
            The train mask
        """


class BootstrapCV(BaseCrossValidator):
    """Cross-validate with bootstrapping.
    The bootstrap CV class makes no guarantees about exclusivity between folds.
    This is simply a naive way to handle KFold cross-validation for something as
    complex as a collaborative filtering split.
    """

    def _iter_train_mask(self, u, i, r):
        """Compute the training mask here."""
        train_size = 1. - (1. / self.n_splits)
        # train_size = 1. - ((n_samples / self.n_splits) / n_samples)
        random_state = check_random_state(self.random_state)

        for split in range(self.n_splits):
            yield _get_stratified_tr_mask(
                u, i, train_size=train_size,
                random_state=random_state.randint(MAX_SEED))

In [6]:
rs = np.random.RandomState(42)
users = LabelEncoder().fit_transform(ratings['user'])
items = LabelEncoder().fit_transform(ratings['item'])
train_data, test_data = train_test_split(users, items, ratings['rating'], random_state=rs)

In [7]:
cv = BootstrapCV(n_splits=3, random_state=42)

In [8]:
steps=300
alpha=0.0002
beta=0.02
score_dict = {"latent":[],"score":[]};
for K in range(10,51,10):
    print('latent' + str(K))
    score = 0
    for j in cv.split(train_data): 
        train, test = j # watch out for K, it shouldn't exceed size of train
        P = np.random.rand(train.shape[0],K)
        Q = np.random.rand(train.shape[1],K)
        Mask_tr = (train!=0)
        Mask_te = (test!=0)
        matrix_estimated,_,_,_ = matrix_factorization_pred(train.todense(),P,Q,K,steps,alpha,beta,Mask_tr.todense())
        extimated_error = np.multiply(test.todense() - matrix_estimated, Mask_te.todense())
        score = score + LA.norm(extimated_error)
    score = score/cv.n_splits
    score_dict["latent"].append(K)
    score_dict["score"].append(score)                

latent10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25

In [9]:
score_dict

{'latent': [10, 20, 30, 40, 50],
 'score': [87.51198472912883,
  82.29717141934985,
  82.66962947347582,
  153.47746847982228,
  153.47638254793472]}